# 分布式计算 总复习

- lect2：PySpark初始化, map & reduce
- lect3：Python中的函数式编程（思想）
- lect4：函数式编程（MapReduce的实现）& 弹幕
- lect5,6：Numpy, RDD & 数值计算原则 
- lect7：ML-（分布式）矩阵乘法&线性回归
- lect8：岭回归、共轭梯度、logistics（理论）

## Lect2：PySpark初始化, map & reduce

### 课堂实例：联合国文字的处理

- PySpark配置和启动
    ```python
    import findspark
    findspark.init()

    from pyspark.sql import SparkSession
    # 本地模式
    spark = SparkSession.builder.master("local[*]").appName("Reading Text").getOrCreate()
    sc = spark.sparkContext
    # sc.setLogLevel("ERROR")
    print(spark)
    print(sc)    
    ```

- PySpark中文件读取，查看，筛选

    `sc.textFile("address")`

    `file.count()`

    `file.take()`

    `file.filter()`

    `file.map()`


### Slides大纲


#### 1. 并行计算基本概念 
- Amdahl 定律：并行加速程度计算公式
  $$S_{latency}(s)=\frac{1}{(1-p)+p/s}$$
  $S$：整体提升倍数，$s$：可并行部分加速比，$p$：可并行部分所占时间的比例

#### 2. 并行计算 vs 分布式计算

#### 3. Apache Spark简介与安装
- Hadoop 分布式计算框架
- HDFS 分布式文件系统
- MapReduce
- Spark：Hadoop改进

### 4. Spark运行模式
- 单机模式
  ```python
  spark = SparkSession.builder.\
  master ("local[*]").\
  appName ("Reading Text").\
  getOrCreate()
  ```
  通过`local[*]`指定所有核心，也可以自行指定
  
- 集群模式


## Lect3：Python中的函数式编程（思想）

### 课堂实例：迭代器、计算方差、随机抽样、MapReduce

- 迭代器的创建与访问：
  - `iter()`
  - `next(IT)`

- 一次遍历计算方差：
  $$(n-1)S=\sum_i (x_i-\bar{x})^2=\sum_i x_i^2-n\bar{x}^2$$

- 随机抽样:
  - `for`循环中一次遍历两个循环
    - `for a,b in zip(A,B):`
  - 遍历的同时获取索引
    - ```python
      for i, (w, v) in enumerate(zip(wvec, vvec)):
      print(f"i is {i}, w is {w}, v is {v}")
      ```

- Reduce:
  - syntax:
    ```python
        import functools
        functools.reduce(FUNCTION,ITER,OPTIONAL)
    ```
    `OPTIONAL` 是一个可选选项，用来设定reduce的初始数值（例如在累乘时要设置初值为1）
        

- Filter:
  - syntax:
    - `filter(IS_FUNCTION,ITER)`

      其中`IS_FUNCTION`是一个需要返回`TRUE/FALSE`的函数

- Map:
  - syntax:
    - `map(FUNCTION,ITER)`

- islice （按照指定长度截断迭代器）:
  - syntax:
    - ```python
        import itertools
        itertools.islice(ITER,NUM)
        ```

- `lambda`函数

### slides 讲解

- 函数式编程
  - 概念、特点
  - 核心思想
  - 迭代器：YOLO！
  - 迭代器全部取出为列表：`lst=list(it)`

## Lect4：函数式编程（MapReduce的实现）& 弹幕

- 样本方差计算
    ```python
    num,sum,sq_sum = data.map(lambda x: (1,x,x*x)).reduce(lambda x,y: (x[0]+y[0],x[1]+y[1],x[2]+y[2]))
    mean = sum / num
    sample_var = (sq_sum+num*mean*mean-2*mean*sum)/(num-1)
    print(sample_var)
    ```

## Lect5,6：Numpy, RDD & 数值计算原则

### 5.1 Numpy

#### 基础操作

- `np.array([1,2,3],[3,4,5])` 定义数列（矩阵）

- `np.linspace(start= 1 , stop=5 , num= 12 )`  生成连续数列

- `np.arange(12)` 生成[0,1,...,12]的array

- `np.reshape(VEC, (3,4))` / `vec.reshape(3,4)`

- `np.ones((3,2))` !不是生成单位矩阵，而是全1矩阵

- `np.zeros((2,3))` 零矩阵

- `np.eyes(5)` 只需要输入一个数字，即可生成对应单位阵

- `np.diag([1,3,5])` 对角矩阵

- `vec[-1]` 表示倒数第1个元素

- `vec[start:end]` 左闭右开

- `mat[:,:2]` 表示全部行，前两列（注意python中下标从1开始）

- `np.random.uniform(low = , high = , size = )` 均匀分布

- `np.random.normal(loc = 2 , scale = 3, size = (2,5))` 正态分布 *loc为均值，scale为标准差（非方差）*

- `np.log()` `np.exp()` 对数与指数

- `np.sum(MAT, axis = 0)` 矩阵的求和，其中`axis=0` 表示对每个列求一个和

- `np.mean(MAT, axis = 1)` 矩阵的平均，其中`axis = 1` 表示对每个行求一个平均


>汇总可以按行或者按列进行，这由`axis`参数决定。（联想到矩阵元素下标$a_{ij}$是先行后列，故这里`axis`是0行1列）0表示运算时第一个维度（行）在变化，1表示运算时第二个维度（列）在变化。再次提醒，Python中以0表示第一个元素！


#### 线性代数

- `MAT.transpose()` 转置

- `MAT_A.dot(MAT_B)` / `np.matmul(MAT_A,MAT_B)` 矩阵乘法

- `np.linalg.inv(MAT)` 矩阵的逆(不推荐)

- `np.linalg.solve(B,d)` 解线性方程组 (返回$B^{-1}d$)

- `evals, evecs = np.linalg.eigh(MAT)` 求特征值，特征向量



### 5.2 RDD

PySpark 初始化

```python
import findspark
findspark.init()

from pyspark.sql import SparkSession
# 本地模式
spark = SparkSession.builder.\
    master("local[*]").\
    appName("PySpark RDD").\
    getOrCreate()
sc = spark.sparkContext
sc.setLogLevel("ERROR")
print(spark)
print(sc)
```

#### RDD的创建

- `DAT = sc.parallelize(VEC)` 将一个数据(i.e. VEC)转化为rdd对象(i.e. DAT)

- `DAT.collect()` 收集所有内容

- `DAT.count()` 返回的实质为RDD的迭代次数

- `DAT.take()` / `DAT.first()` 取RDD前几个next的内容



#### RDD 对.txt文件的处理

- `file = sc.textFile("ADDRESS")` 读入文本文件

- `print(*(file.take(5)), sep="\n")` 对前五行的打印

- 将string类型数据转换成numpy数据：
    ```python
    # str => np.array
    def str_to_vec(line):
        # 分割字符串
        str_vec = line.split("\t")
        # 让 Numpy 进行类型转换
        return np.array(str_vec, dtype=float)
    DAT = file.map(str_to_vec)    
    ``` 



#### RDD的分区

- `file.getNumPartitions()` 获得自动分区数

- `file.repartition(NUM)` 自定义分区数

- 对于切分完的分区，我们希望把一个分区中的数据按照一个矩阵来理解，故：

    ```python
    # Iter[str] => Iter[matrix]
    def part_to_mat(iterator):
        # Iter[str] => Iter[np.array]
        iter_arr = map(str_to_vec, iterator)


        # Iter[np.array] => list(np.array)
        dat = list(iter_arr) #dat是由向量（nparray）作为元素组成的列表（list）

    # 有的分区可能是空分区
        # list(np.array) => matrix
        if len(dat) < 1:  # Test zero iterator
            mat = np.array([])
        else:
            mat = np.vstack(dat) 

        # matrix => Iter[matrix]
        yield mat # yield可以认为是return[mat]，返回的不是mat本身，而是包含这个的容器

    dat_p10 = file_p10.mapPartitions(part_to_mat)

    dat_p10_nonempty = dat_p10.filter(lambda x: x.shape[0] > 0)

    ```



### 5.3 数值计算原则

#### 原则1：矩阵相乘，小维度优先

- 经验法则：对于更一般的矩阵乘法 $A_{m\times n}B_{n\times p}C_{p\times r}$，如果 $n\approx p$ 且 $m>r$，则优先计算 $BC$，反之优先计算 $AB$。

- 对于一个$A_{n\times p }\times B_{p\times q}$的矩阵乘法，时间复杂度为$O(npq)$

#### 原则2：尽量避免显式矩阵求逆

常见矩阵运算复杂度整理：

（假设$A,B : n\times n, b :n\times1$）

矩阵乘法：
- $AB:O(n^3)$
- $Ab:O(n^2)$

矩阵的逆：
- $A^{-1}:O(n^3)$
- $A^{-1}b:O(n^3)$ （但从实证方面看比求逆效率更高）

矩阵的其他运算：
- $|A|, eign(A) :O(n^3) $
- 特别的：上/下三角矩阵的行列式$O(n)$ （行列式为对角线的乘积）
- $||A||_p^2 : O(n^2)$
- $ A+b1^T : O(n^2)$ 相当于将$b$广播



#### 原则3：利用矩阵的特殊结构

- 对于对角矩阵，在存储时应当以向量存储对角线元素即可，事实上在参与计算中也可以通过广播机制直接以向量参与运算：
  - 假设$D=diag(d_1,...,d_n)$为对角线元素，$A$为正常矩阵，则$DA$相当于对第i列乘以$d_i$，$AD$相当于对第i行乘以$d_i$

#### 原则4：尽可能将显示循环转化为矩阵计算

- 虽然理论复杂度相似，但实证上，考虑通信成本等，矩阵运算更快

In [ ]:
# 1. 初始化pyspark

import findspark
findspark.init("/Users/xinby/Library/Spark")

from pyspark.sql import SparkSession
# 本地模式
spark = SparkSession.builder.\
    master("local[*]").\
    appName("PySpark RDD").\
    getOrCreate()
sc = spark.sparkContext
sc.setLogLevel("ERROR")
print(spark)
print(sc)

# 2. 生成数据

import numpy as np
np.set_printoptions(linewidth=100)

np.random.seed(123)
n = 100
p = 5
mat = np.random.normal(size=(n, p))
np.savetxt("mat_np.txt", mat, fmt="%f", delimiter="\t")

# 3. 读取数据至RDD并分区

file = sc.textFile("mat_np.txt")
file_p10 = file.repartition(10)
# str => np.array
def str_to_vec(line):
    # 分割字符串
    str_vec = line.split("\t")
    # 将每一个元素从字符串变成数值型
    num_vec = map(lambda s: float(s), str_vec)
    # 创建 Numpy 向量
    return np.fromiter(num_vec, dtype=float)

# Iter[str] => Iter[matrix]
def part_to_mat(iterator):
    # Iter[str] => Iter[np.array]
    iter_arr = map(str_to_vec, iterator)

    # Iter[np.array] => list(np.array)
    dat = list(iter_arr)

    # list(np.array) => matrix
    if len(dat) < 1:  # Test zero iterator
        mat = np.array([])
    else:
        mat = np.vstack(dat)

    # matrix => Iter[matrix]
    yield mat

dat = file_p10.mapPartitions(part_to_mat).filter(lambda x: x.shape[0] > 0)

**原理**

假设：$ X \in \R^{n\times p}, v \in \R^{p}$

将$X$按照行进行分块（包含所有列，但行不一定是一行），记为：$X=[X_1;...;X_m]^T$, $X_i \in \R^{n_i\times p}$

故$Xv=[X_1v;...;X_mv]^T$

In [ ]:
np.random.seed(123)
v = np.random.uniform(size=p)

# 这里不是reduce是因为不是要把最后分块进行加和，而是想要最后的拼接
res_part = dat.map(lambda x: x.dot(v)).collect() 
print(res_part,type(res_part))
# 这个结果里，一个array就是前面的一个分块
np.concatenate(res_part)

**原理**

同理进行分块，则最终的结果为：$X'X=X_1'X_1+\dots+X_m'X_m$ (由于这里的假定$X\in\R^{n\times p}, n>>p$，故最终的大小为$p\times p$，是可以存储在内存中的)

In [ ]:
res = dat.map(lambda x: x.T.dot(x)).reduce(lambda x, y: x + y) 
# x代表了前面的累积结果，y代表了最新一项

## Lect 7：ML-（分布式）矩阵乘法&线性回归（OLS）

### 7.1 矩阵乘法（RDD）

#### 0. *准备工作*

#### 1. 矩阵乘法$Xv$

#### 2. 矩阵乘法 $X'X$

#### 3. 矩阵乘法 $X'v$

**原理**

这里设定$X,v$的行数相同，故可以同时进行拆分(在具体操作时认为是在一个大矩阵中)，$X'v = X_1'v_1+...+X_m'v_m$

In [ ]:
X = mat[:, :-1]
v = mat[:, -1]
def Xitv(part):
    '''
    在定义这个map函数的时候，由于整体是一个大矩阵，但是在内部需要将X和v区分开来
    '''
    Xi = part[:, :-1]
    vi = part[:, -1]
    return Xi.transpose().dot(vi)

res = dat.map(Xitv).reduce(lambda x, y: x + y)

### 7.2 线性回归（OLS） （RDD）

**理想OLS的理论模型与假设：**

$$y = \beta_0 + \beta'x +\epsilon$$

其中 $X\in \R^{n\times p}$，这里要求$n>>p$

$$\hat\beta = (X'X)^{-1}X'y$$

- 若要包含截距项，则$X$的第一列为1

**计算过程**
1. 首先计算$X'X,X'y$
2. 然后计算$(X'X)^{-1}X'y$
   
事实上，由矩阵分块原理，可以只计算一次：$X'[X;y]$


#### 实例：RDD实现 （hw4）

##### 参数估计

In [ ]:
xt_xy = data_partition_nonempty.\
    map(lambda x: np.hstack((np.ones((np.shape(x)[0],1)),x))).\
    map(lambda x: x[:,:-1].transpose().dot(x) ).\
    reduce (lambda x,y: x+y)

xt_x = xt_xy[:,:-1]
xt_y = xt_xy[:,-1]

hat_beta = np.linalg.solve(xt_x,xt_y)
print(hat_beta)

##### R_sq

相关公式：
$$SSR = \sum (y_i-\hat y_i)^2 = ||Y-X\hat\beta||_2$$
$$SST = \sum (y_i - \bar y)^2 = \sum y_i^2+n\bar y^2-2n\bar y \sum y_i $$
$$ R^2 = 1 - SSR/SST$$

假设：
1. 已知$\hat\beta, [X,Y]$
2. 数据经过mapPartition在多个矩阵中存储
   
计算过程：
1. 扩充$X := [1,X]$
2. 计算 $Y-X\hat\beta$，稍后对其进行平方求和
3. 计算 $\sum y_i, \sum y_i^2$
4. reduce，根据上述Rsquare公式进行整合计算

In [ ]:
sum_y, sum_y_sq, ssr, num = data_partition_nonempty.\
    map(lambda x: np.hstack((np.ones((np.shape(x)[0],1)),x))).\
    map(lambda x: (np.sum(x[:,-1]),np.sum(x[:,-1]**2),np.sum(x[:,-1]-x[:,:-1].dot(hat_beta)**2,axis=0),np.shape(x)[0])).\
    reduce(lambda x,y:(x[0]+y[0],x[1]+y[1],x[2]+y[2],x[3]+y[3]) )
print(sum_y,sum_y_sq, ssr, num)
y_bar = sum_y/num
sst = sum_y_sq + n*y_bar**2 -2*num*y_bar*sum_y
R_sq = 1 - ssr / sst
print(f"R^2: {R_sq}")


## Lect8：岭回归、共轭梯度法、logistics（理论）

### 8.1 岭回归

**理论**

- 当 $n<p$ 时，$X'X$ 不可逆，此时最小二乘**没有唯一解** （事实上是有无数组解使得$\min S_c=0$，即可以完美拟合）【并不是说不存在解！！】
- 此时的OLS并不影响预测性，但是并不影响解释性
- 此时我们可以采用岭回归的方法，其在最小二乘损失函数的基础上加入一个惩罚项 

$$Loss = ||Y-X\beta||_2+\lambda ||\beta||^2$$

$$\hat\beta_\lambda = (X^TX+\lambda I)^{-1}X^TY$$

- 但注意到 $X'X+\lambda I$ 是一个高维的矩阵($p\times p$)，难以直接进行求解。因此我们采用共轭梯度法

**Python 实现**

### 8.2 共轭梯度法

**算法目的**

求解$Ax=b$（只要$A_{n \times n}$是正定的，n步内即求得精确解）

- 时间复杂度最差为$O(n^3)$

**CG 的python实现**

```python
def cg(A, b, x0, eps=1e-3, print_progress=False):
    ''' 已知Ax0=b，求解x
    A: n*n
    b: n*1
    x0: n*1
    eps: 精度
    print_progress: 是否打印迭代过程
    '''
    m = b.shape[0]
    # 初始解（注意此处应该复制x0，否则程序退出时会修改x0）
    x = np.copy(x0)
    # 初始残差向量
    r = b - np.dot(A, x)
    # 初始共轭梯度
    p = r

    for k in range(m):
        # 矩阵乘法
        Ap = np.dot(A, p)
        rr = r.dot(r)
        alpha = rr / p.dot(Ap)
        # 更新解
        x += alpha * p
        # 计算新残差向量
        rnew = r - alpha * Ap
        # 测试是否收敛
        norm = np.linalg.norm(rnew)
        if print_progress:
            print(f"Iter {k}, residual norm = {norm}")
        if norm < eps:
            break
        beta = rnew.dot(rnew) / rr
        # 更新共轭梯度
        p = rnew + beta * p
        # 更新残差向量
        r = rnew

    return x
```